# Final Composition
In this notebook I shall compose all the methods that I worked on so far and develop the final solution. A LOT of tinkering and moving around will be done, and some new features might be added, but I'll likely skip long sections for intermediate steps and directly include it in the document. 

In [55]:
# Set the right working folder to root folder of the project
import os

print("Looking for root folder of the project...")
for folder_depth in range(100): 
    if os.path.exists(".git"):
        root_folder = os.getcwd()
        print("Root folder found. Now working in directory '%s'" % os.getcwd())
        break
    else:
        print("Going up from '%s'" % os.getcwd())
        os.chdir("..")
else:
    raise Exception("Root folder of the project not found. Terminating.")
    

Looking for root folder of the project...
Root folder found. Now working in directory 'D:\Linas\projects\CarND-Advanced-Lane-Lines'


In [78]:
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import clips_array
from moviepy.editor import ipython_display

import cv2
import numpy as np
from math import ceil


class Plotter:
    def __init__(self, columns, figsize=(20, 40)):
        plt.figure(figsize=figsize)
        
        self.columns = columns
        self.images = []
        self.extra_plots = []
        
    def add_img(self, img, title):
        assert len(img.shape) == 2 or img.shape[2] == 3
        
        cmap = None
        if len(img.shape) == 2:
            cmap = "gray"

        self.images.append((img, title, cmap))
        self.extra_plots.append([])
        
    def add_extra_to_last_img(self, xs, ys, plot_type):
        assert plot_type in ["line"]
        self.extra_plots[-1].append((xs, ys, plot_type))
            
    def plot(self):
        j = 1  # Current column
        i = 0
        rows = ceil(len(self.images) // self.columns)
        for img_index, (img, title, cmap) in enumerate(self.images):
            # Draw iamge
            ax = plt.subplot((rows / self.columns + 1) * self.columns, self.columns, i * self.columns + j)
            ax.set_title(title)
            
            # Draw Extras
            for xs, ys, plot_type in self.extra_plots[img_index]:
                if plot_type == "line":
                    plt.plot(xs, ys, color='yellow')
            
            # Show
            plt.imshow(img, cmap="gray")
            
            if j % self.columns == 0:
                j = 0
                i += 1
            j += 1


In [79]:
YM_PER_PX = 30 / 720
XM_PER_PX = 3.7 / 850  # empirically found that horizontal distance between lines is 850 pixels


def apply_precomputed_undistortion(img, mtx_filename, dist_filename):
    mtx = np.load(mtx_filename)
    dist = np.load(dist_filename)
    undist_img = cv2.undistort(img, mtx, dist)
    return undist_img


def threshold_image(img):
    # Saturation-based thresholding
    hls_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls_img[:, :, 2]
    
    s_thresh_value = 150
    s_thresh = np.zeros_like(s_channel)
    s_thresh[s_channel > s_thresh_value] = 1
    
    # Edge Detection
    red = img[:, :, 0]
    sobel_kernel = 25
    assert sobel_kernel >= 3 and sobel_kernel % 2 == 1
    sobel_x = cv2.Sobel(red, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(red, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    sobel_x = np.absolute(sobel_x)
    sobel_y = np.absolute(sobel_y)

    # Magnitude
    magnitude = (sobel_x**2 + sobel_y**2)**.5
    mag_norm = np.uint8(255 * magnitude / np.max(magnitude))
    mag_thresh_value = 30
    mag_thresh = np.zeros_like(mag_norm, dtype=np.uint8)
    mag_thresh[mag_norm > mag_thresh_value] = 1
    
    # Direction
    atan = np.arctan2(sobel_y, sobel_x)
    atan_thresh_min = 0.8
    atan_thresh_max = 1.2
    dir_thresh = np.zeros_like(atan, dtype=np.uint8)
    dir_thresh[(atan > atan_thresh_min) & (atan <= atan_thresh_max)] = 1
    
    # Combinations
    mag_and_dir = cv2.bitwise_and(mag_thresh, dir_thresh)    
    grad_or_color = cv2.bitwise_or(mag_and_dir, s_thresh)
    
    return grad_or_color


def warp_perspective(img, x_offset=200, y_offset=100, new_image_shape=(1000, 1000), flags=cv2.INTER_LINEAR):    
    # Detect Lines
    # Luckily, the camera resolution is 1280 x 720 in all iamges / videos we're given. This means I can hardode the values.
    top_left  = [578,  463]
    top_right = [706,  463]
    bot_right = [1043, 677]
    bot_left  = [267,  677]
    src_corners = np.float32([top_left, top_right, bot_right, bot_left])
    
    line_img = np.zeros(new_image_shape, dtype=np.uint8)
    
    new_top_left  = [x_offset, y_offset]
    new_top_right = [line_img.shape[1] - x_offset, y_offset]
    new_bot_right = [line_img.shape[1] - x_offset, line_img.shape[0] - y_offset]
    new_bot_left  = [x_offset, line_img.shape[0] - y_offset]
    dst_corners = np.float32([new_top_left, new_top_right, new_bot_right, new_bot_left])
    
    transform_matrix = cv2.getPerspectiveTransform(src_corners, dst_corners)
    inv_transform_matrix = cv2.getPerspectiveTransform(dst_corners, src_corners)
    warped = cv2.warpPerspective(img, transform_matrix, line_img.shape[::-1], flags=flags)

    return warped, transform_matrix, inv_transform_matrix


def retrieve_polylines(warped, draw_windows=True):    
    """
    Takes in the warped thresholds to find polylines on the road
    """
    
    # find histogram of half-height
    warped_y_midpoint = warped.shape[0] // 2
    histogram = np.sum(warped[warped_y_midpoint:, :], axis=0)
    out_img = np.dstack((warped, warped, warped))
    
    # find base points on both sides - max values
    hist_x_midpoint = np.int(histogram.shape[0] // 2)
    left_x_base = np.argmax(histogram[:hist_x_midpoint])
    right_x_base = np.argmax(histogram[hist_x_midpoint:]) + hist_x_midpoint

    # Hyperparameters
    win_min_pix = 50  # minimum number of pixels found to recenter window
    win_num = 9  # the number of sliding windows
    win_half_width = 200 // 2
    win_height = np.int(warped.shape[0] // win_num)
    
    # Nonzero pixel indices
    nonzero = warped.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])
    
    # Init current midpoint variables and index lists
    left_x_current = left_x_base
    right_x_current = right_x_base
    left_lane_inds = []
    right_lane_inds = []
    
    for win_index in range(win_num):
        # Iterate over windows from the bottom, setting boundaries
        win_y_low = warped.shape[0] - (win_index + 1) * win_height
        win_y_high = warped.shape[0] - win_index * win_height
        win_x_left_low   = left_x_current - win_half_width
        win_x_left_high  = left_x_current  + win_half_width
        win_x_right_low  = right_x_current - win_half_width
        win_x_right_high = right_x_current + win_half_width
        
        # Draw the windows on the visualization image
        if draw_windows:
            cv2.rectangle(out_img, (win_x_left_low,  win_y_low), (win_x_left_high,  win_y_high), color=(0, 255, 0), thickness=3)
            cv2.rectangle(out_img, (win_x_right_low, win_y_low), (win_x_right_high, win_y_high), color=(0, 255, 0), thickness=3)
        
        # Conjoin binary index arrays, retrieve y coords of nonzero pixels in each window
        good_left_inds = (
            (nonzero_x >= win_x_left_low) &
            (nonzero_x < win_x_left_high) &
            (nonzero_y >= win_y_low) &
            (nonzero_y < win_y_high)
        ).nonzero()[0]
        good_right_inds = (
            (nonzero_x >= win_x_right_low) &
            (nonzero_x < win_x_right_high) &
            (nonzero_y >= win_y_low) &
            (nonzero_y < win_y_high)
        ).nonzero()[0]
        
        # Append whole array to a global list
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If enough pixels, shift mean x coordinate of the 
        if len(good_left_inds) > win_min_pix:
            nonzero_x_in_win = nonzero_x[good_left_inds]
            left_x_current = np.int(np.mean(nonzero_x_in_win))
        if len(good_right_inds) > win_min_pix:
            nonzero_x_in_win = nonzero_x[good_right_inds]
            right_x_current = np.int(np.mean(nonzero_x_in_win))

    # Flatten global list to get all y indices of line pixels for each side
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError as e:
        raise e

    # Get all (i,j) indices of nonzero pixels within windows, for each side 
    left_x = nonzero_x[left_lane_inds]
    left_y = nonzero_y[left_lane_inds] 
    right_x = nonzero_x[right_lane_inds]
    right_y = nonzero_y[right_lane_inds]
    
    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[left_y, left_x] = [255, 0, 0]
    out_img[right_y, right_x] = [0, 0, 255]
    
    # Polyfit using all the points
    left_fit = np.polyfit(left_y, left_x, 2)
    right_fit = np.polyfit(right_y, right_x, 2)
    
    # Get real-scaled polylines for curvature calculation
    scaled_left_fit = np.polyfit(left_y * YM_PER_PX, left_x * XM_PER_PX, 2)
    scaled_right_fit = np.polyfit(right_y * YM_PER_PX, right_x * XM_PER_PX, 2)

    return out_img, left_fit, right_fit, (scaled_left_fit, scaled_right_fit)


def retrieve_polylines_from_older(warped, left_fit, right_fit, draw_windows=True):
    """
    The same polyfit function, but this time it uses the window around previous polynomial lines to search for pixels.
    """
    # Hyperparameters
    win_half_width = 200 // 2

    # Nonzero pixel indices
    nonzero = warped.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])
    
    # Instead of rectangular windows, find binary arrays with nonzero pixels around given polyfit lines
    left_fit_x = left_fit[0] * nonzero_y**2 + left_fit[1] * nonzero_y + left_fit[2]
    left_lane_inds = (
        (nonzero_x > (left_fit_x - win_half_width)) &
        (nonzero_x < (left_fit_x + win_half_width))
    )
    right_fit_x = right_fit[0] * (nonzero_y**2) + right_fit[1] * nonzero_y + right_fit[2]
    right_lane_inds = (
        (nonzero_x > (right_fit_x - win_half_width)) &
        (nonzero_x < (right_fit_x + win_half_width))
    )

    # Get all (i,j) indices of nonzero pixels within line-based windows, for each side 
    left_x = nonzero_x[left_lane_inds]
    left_y = nonzero_y[left_lane_inds]
    right_x = nonzero_x[right_lane_inds]
    right_y = nonzero_y[right_lane_inds]

    # Polyfit using all the points
    left_fit = np.polyfit(left_y, left_x, 2)
    right_fit = np.polyfit(right_y, right_x, 2)
    
    # Generate list [0, 1, ..., warped.shape[0] - 1]  (along y coordinates)
    plot_y = np.linspace(0, warped.shape[0] - 1, warped.shape[0])
    
    # Generate new left and right fit x coordinates
    left_fit_x  = left_fit[0]  * plot_y**2 + left_fit[1]  * plot_y + left_fit[2]
    right_fit_x = right_fit[0] * plot_y**2 + right_fit[1] * plot_y + right_fit[2]
    
    # Prepare output and line image
    out_img = np.dstack((warped, warped, warped)) * 255
    line_win_img = np.zeros_like(out_img)

    # Color all pixels in line-based windows red and blue
    out_img[nonzero_y[left_lane_inds],  nonzero_x[left_lane_inds]]  = [255, 0, 0]
    out_img[nonzero_y[right_lane_inds], nonzero_x[right_lane_inds]] = [0, 0, 255]

    # 2 layers [column vector of left_fit_x ± win_width, column of [0,1,...,height]]
    left_line_window_1 = np.array([
        np.vstack([left_fit_x - win_half_width, plot_y]).T
    ])    
    left_line_window_2 = np.array([
        np.flipud(np.vstack([left_fit_x + win_half_width, plot_y]).T)
    ])
    left_line_pts = np.hstack((left_line_window_1, left_line_window_2))
    
    # 2 layers [column vector of right_fit_x ± win_width, column of [0,1,...,height]]
    right_line_window_1 = np.array([
        np.vstack([right_fit_x - win_half_width, plot_y]).T
    ])
    right_line_window_2 = np.array([
        np.flipud(np.vstack([right_fit_x + win_half_width, plot_y]).T)
    ])
    right_line_pts = np.hstack((right_line_window_1, right_line_window_2))

    if draw_windows:
        # Fill polies within windows
        cv2.fillPoly(line_win_img, np.int_([left_line_pts]), (0, 150, 0))
        cv2.fillPoly(line_win_img, np.int_([right_line_pts]), (0, 150, 0))
        result = cv2.addWeighted(out_img, 1, line_win_img, 0.3, 0)
    else:
        result = out_img
    
    return result, left_fit, right_fit, None

def generate_polyline_plots(warped_shape, left_fit, right_fit):
    # Generate x and y values for plotting
    plot_y = np.linspace(0, warped_shape[0] - 1, warped_shape[0])

    left_fit_x = left_fit[0] * plot_y**2 + left_fit[1] * plot_y + left_fit[2]
    right_fit_x = right_fit[0] * plot_y**2 + right_fit[1] * plot_y + right_fit[2]

    # # Now call:
    # plotter.add_img(out_img, "warped image")
    # plotter.add_extra_to_last_img(left_fit_x,  plot_y, "line")
    # plotter.add_extra_to_last_img(right_fit_x, plot_y, "line")
    
    return left_fit_x, right_fit_x, plot_y
    
def fit_polynomial(x, coefficients):
    res = 0
    for power, coeff in enumerate(coefficients[::-1]):
        res += coeff * x**power
    return res

def get_lane_area(warped_line_img, inv_transformation_matrix, original_img_shape, left_fit, right_fit):
    # Area between two polylines
    all_pts = np.zeros(warped_line_img.shape[:2], dtype=np.uint8)
    for y in range(warped_line_img.shape[0]):
        poly_left = int(fit_polynomial(y, left_fit))
        poly_right = int(fit_polynomial(y, right_fit))
        all_pts[y, poly_left:poly_right] = 1
    
    polyfilled = warped_line_img.copy()
    polyfilled[all_pts == 1] = np.array([0, 255, 0])
    
    # Dewarp
    dewarped = cv2.warpPerspective(polyfilled, inv_transformation_matrix, original_img_shape[1::-1], flags=cv2.INTER_LINEAR)
    
    return dewarped

In [80]:

def retrieve_polylines_from_several_images(warped_images, draw_windows=True):
    """
    Takes in the warped thresholds to find polylines on the road
    """
    fused_img = np.zeros(warped_images.shape[:2], dtype=np.uint8)
    for img_index in range(warped_images.shape[2]):
        fused_img = cv2.bitwise_or(fused_img, warped_images[:, :, img_index])
    
    # find histogram of half-height
    warped_y_midpoint = warped_images.shape[0] // 2
    histogram = np.sum(warped_images[warped_y_midpoint:, :, :], axis=0)
    out_img = np.dstack((fused_img, fused_img, fused_img))
     
    # find base points on both sides - max values
    hist_x_midpoint = np.int(histogram.shape[0] // 2)
    left_x_base = np.argmax(histogram[:hist_x_midpoint])
    right_x_base = np.argmax(histogram[hist_x_midpoint:]) + hist_x_midpoint

    # Hyperparameters
    win_min_pix = 50  # minimum number of pixels found to recenter window
    win_num = 9  # the number of sliding windows
    win_half_width = 200 // 2
    win_height = np.int(warped_images.shape[0] // win_num)
    
    # TODO: finish this   
 
#     # Nonzero pixel indices
#     nonzero_x_list = []
#     nonzero_y_list = []
#     for img_index in range(warped_images.shape[2]):
#         nonzero = warped_images[:, :, img_index].nonzero()
#         print("sub: ", np.array(nonzero[0]).shape)
#         nonzero_y_list.append(np.array(nonzero[0]))
#         nonzero_x_list.append(np.array(nonzero[1]))
#     nonzero_y = np.concatenate(nonzero_y_list)
#     nonzero_x = np.concatenate(nonzero_x_list)
    
#     print(nonzero_y.shape)
#     print(nonzero_x.shape)
    
#     # Init current midpoint variables and index lists
#     left_x_current = left_x_base
#     right_x_current = right_x_base
#     left_lane_inds = []
#     right_lane_inds = []
    
#     for win_index in range(win_num):
#         # Iterate over windows from the bottom, setting boundaries
#         win_y_low = warped.shape[0] - (win_index + 1) * win_height
#         win_y_high = warped.shape[0] - win_index * win_height
#         win_x_left_low   = left_x_current - win_half_width
#         win_x_left_high  = left_x_current  + win_half_width
#         win_x_right_low  = right_x_current - win_half_width
#         win_x_right_high = right_x_current + win_half_width
        
#         # Draw the windows on the visualization image
#         if draw_windows:
#             cv2.rectangle(out_img, (win_x_left_low,  win_y_low), (win_x_left_high,  win_y_high), color=(0, 255, 0), thickness=3)
#             cv2.rectangle(out_img, (win_x_right_low, win_y_low), (win_x_right_high, win_y_high), color=(0, 255, 0), thickness=3)
        
#         # Conjoin binary index arrays, retrieve y coords of nonzero pixels in each window
#         good_left_inds = (
#             (nonzero_x >= win_x_left_low) &
#             (nonzero_x < win_x_left_high) &
#             (nonzero_y >= win_y_low) &
#             (nonzero_y < win_y_high)
#         ).nonzero()[0]
#         good_right_inds = (
#             (nonzero_x >= win_x_right_low) &
#             (nonzero_x < win_x_right_high) &
#             (nonzero_y >= win_y_low) &
#             (nonzero_y < win_y_high)
#         ).nonzero()[0]
        
#         # Append whole array to a global list
#         left_lane_inds.append(good_left_inds)
#         right_lane_inds.append(good_right_inds)
        
#         # If enough pixels, shift mean x coordinate of the 
#         if len(good_left_inds) > win_min_pix:
#             nonzero_x_in_win = nonzero_x[good_left_inds]
#             left_x_current = np.int(np.mean(nonzero_x_in_win))
#         if len(good_right_inds) > win_min_pix:
#             nonzero_x_in_win = nonzero_x[good_right_inds]
#             right_x_current = np.int(np.mean(nonzero_x_in_win))

#     # Flatten global list to get all y indices of line pixels for each side
#     try:
#         left_lane_inds = np.concatenate(left_lane_inds)
#         right_lane_inds = np.concatenate(right_lane_inds)
#     except ValueError as e:
#         raise e

#     # Get all (i,j) indices of nonzero pixels within windows, for each side 
#     left_x = nonzero_x[left_lane_inds]
#     left_y = nonzero_y[left_lane_inds] 
#     right_x = nonzero_x[right_lane_inds]
#     right_y = nonzero_y[right_lane_inds]
    
#     ## Visualization ##
#     # Colors in the left and right lane regions
#     out_img[left_y, left_x] = [255, 0, 0]
#     out_img[right_y, right_x] = [0, 0, 255]
    
#     # Polyfit using all the points
#     left_fit = np.polyfit(left_y, left_x, 2)
#     right_fit = np.polyfit(right_y, right_x, 2)

    left_fit = [1, 1, 1]
    right_fit = [1, 2, 3]

    return out_img, left_fit, right_fit, None

In [56]:
def threshold_yellow_lines(hls_img):
    """
    This function attempts to detect only the yellow lines.
    
    Why try to make a general function for detecting all lane lines, when we can easily distinguish two types - yellow and white?
    (actually, there's a third type, when road side is used as a line, with no explicit line marking)
    
    I have consulted https://driversed.com/driving-information/signs-signals-and-markings/markings-colors-patterns-meaning.aspx,
    verifying that only two color types exist.
    """
    # Convert to hls
    h_channel = hls_img[:, :, 0]
    l_channel = hls_img[:, :, 1]
    s_channel = hls_img[:, :, 2]
    
    thresh = np.zeros_like(h_channel)
    thresh[
        (h_channel >= 11) & (h_channel <= 30) &
        (s_channel >= 50) &
        (l_channel >= 150)
    ] = 255
    
    return thresh


def threshold_strong_sat_light(hls_img):
    h_channel = hls_img[:, :, 0]
    l_channel = hls_img[:, :, 1]
    s_channel = hls_img[:, :, 2]
    
    thresh = np.zeros_like(h_channel)
    thresh[
        (s_channel >= 150) &
        (l_channel >= 128)
    ] = 255

    return thresh


def apply_sobel(channel, kernel_size):
    assert kernel_size >= 3 and kernel_size % 2 == 1
    sobel_x = cv2.Sobel(channel, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobel_y = cv2.Sobel(channel, cv2.CV_64F, 0, 1, ksize=kernel_size)
    
    sobel_x = np.absolute(sobel_x)
    sobel_y = np.absolute(sobel_y)
    
    return sobel_x, sobel_y


def threshold_magnitude(sobel_x, sobel_y):
    magnitude = (sobel_x**2 + sobel_y**2)**.5
    mag_norm = np.uint8(255 * magnitude / np.max(magnitude))
    mag_thresh_value = 30
    mag_thresh = np.zeros_like(mag_norm, dtype=np.uint8)
    mag_thresh[mag_norm > mag_thresh_value] = 255
    
    return mag_thresh


def threshold_direction(sobel_x, sobel_y):
    atan = np.arctan2(sobel_y, sobel_x)  # returns results in range [0.0, 1.57079632679]
    atan_thresh_max = 1.2
    dir_thresh = np.zeros_like(atan, dtype=np.uint8)
    dir_thresh[(atan <= atan_thresh_max)] = 255
    
    return dir_thresh


def morph_close(thresh, kernel_size):
    assert kernel_size >= 3 and kernel_size % 2 == 1
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    return closed
    
    
def get_distorted_hood_mask():
    """
    Returns the original image mask with the hood area masked
    """
    mask_filename = os.path.join(root_folder, "masks", "distorted_hood_mask.png")
#     mask_rgb = mpimg.imread(mask_filename)
    mask_bgr = cv2.imread(mask_filename)
    
    
    mask = np.zeros(mask_bgr.shape[:2], dtype=np.uint8)
    mask[
        (mask_bgr[:, :, 0] == 255) & 
        (mask_bgr[:, :, 1] == 255) & 
        (mask_bgr[:, :, 2] == 0)
    ] = 255
    mask = cv2.bitwise_not(mask)
                    
    return mask


def compute_curvature(max_y, scaled_left_fit, scaled_right_fit):
    # Calculation of R_curve (radius of curvature)
    left_curverad  = ((1 + (2 * scaled_left_fit[0] * max_y * YM_PER_PX  + scaled_left_fit[1]) ** 2) ** 1.5)  / np.absolute(2 * scaled_left_fit[0])
    right_curverad = ((1 + (2 * scaled_right_fit[0] * max_y * YM_PER_PX + scaled_right_fit[1]) ** 2) ** 1.5) / np.absolute(2 * scaled_right_fit[0])
    
    return left_curverad, right_curverad
    

    

In [77]:
def make_color(func):
    """
    Converts the result to a color image if a grayscale is provided
    """
    def wrapper(*args, **kwargs):
        img = func(*args, **kwargs)
        if len(np.shape(img)) == 2:
            return cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        else:
            return img
        
    return wrapper



@make_color
def process_img(rgb_img,
                left_lines, right_lines, line_history_length,
                prev_left_holder, prev_right_holder,
                scaled_left_lines, scaled_right_lines,
                left_line_centerpoints, right_line_centerpoints):
    # Get hood mask to ignore hood area
    dist_hood_mask = get_distorted_hood_mask()
        
    # Undistort Image
    mtx_filename  = os.path.join(root_folder, "params", "camera_matrix.npy")
    dist_filename = os.path.join(root_folder, "params", "dist_coeffs.npy")
    undistorted = apply_precomputed_undistortion(rgb_img, mtx_filename, dist_filename)
    hood_mask = apply_precomputed_undistortion(dist_hood_mask, mtx_filename, dist_filename)    
    hood_mask[hood_mask < 100] = 0     # Undo the effects of linear interpollation
    hood_mask[hood_mask >= 100] = 255

    # RGB -> HLS
    hls_img = cv2.cvtColor(undistorted, cv2.COLOR_RGB2HLS)
    h_channel = hls_img[:, :, 0]
    l_channel = hls_img[:, :, 1]
    s_channel = hls_img[:, :, 2]
    
    # Find x and y edges
    l_sobel_x, l_sobel_y = apply_sobel(l_channel, 25)
    s_sobel_x, s_sobel_y = apply_sobel(s_channel, 25)
    
#     # Threshold by color
#     thresh_yellow = threshold_yellow_lines(hls_img)
#     thresh_sat_light = threshold_strong_sat_light(hls_img)

    # Threshold by edge magnitude
    l_thresh_mag = threshold_magnitude(l_sobel_x, l_sobel_y)
    s_thresh_mag = threshold_magnitude(s_sobel_x, s_sobel_y)
    thresh_mag = cv2.bitwise_or(l_thresh_mag, s_thresh_mag)
    
    # Threshold by edge direction
    l_thresh_dir = threshold_direction(l_sobel_x, l_sobel_y)  # I results are slightly better with just lightness-based threshold than both.
    
    # Combine edge magnitude and direction
    thresh_edges = cv2.bitwise_and(thresh_mag, l_thresh_dir)

    # Fill in gaps between edges
    closed_edges = morph_close(thresh_edges, 5)
    
    # Apply hood mask
    closed_edges = cv2.bitwise_and(closed_edges, hood_mask)

#     thresh_yellow = cv2.bitwise_and(thresh_yellow, hood_mask)
#     thresh_sat_light = cv2.bitwise_and(thresh_sat_light, hood_mask)
    
#     # Fuse all thresholds
#     thresh_color = cv2.bitwise_or(thresh_yellow, thresh_sat_light)
#     final_thresh = cv2.bitwise_and(closed_edges, thresh_color)
    final_thresh = closed_edges
    
    warped, transformation_matrix, inv_transformation_matrix = warp_perspective(
        final_thresh, x_offset=200, y_offset=100, new_image_shape=closed_edges.shape[:2]
    )
    
    # Focus on the road lines by warping perspective
#     warped_edges, transformation_matrix, inv_transformation_matrix = warp_perspective(
#         closed_edges, x_offset=200, y_offset=100, new_image_shape=closed_edges.shape[:2]
#     )
#     warped_yellows, _, _ = warp_perspective(
#         thresh_yellow, x_offset=200, y_offset=100, new_image_shape=thresh_yellow.shape[:2]
#     )
#     warped_whites, _, _ = warp_perspective(
#         thresh_sat_light, x_offset=200, y_offset=100, new_image_shape=thresh_sat_light.shape[:2]
#     )
    
#     # Fit a polyline
    warped_line_img, left_fit, right_fit, scaled_lines = retrieve_polylines(warped, draw_windows=False)
    
# #     if prev_left_holder[0] is None or prev_right_holder[0] is None:
# #         warped_line_img, left_fit, right_fit, _ = retrieve_polylines(warped_edges, draw_windows=False)
# #     else:
# #         warped_line_img, left_fit, right_fit, _ = retrieve_polylines_from_older(
# #             warped_edges, prev_left_holder[0], prev_right_holder[0], draw_windows=False
# #         )

# #     warped_line_img, left_fit, right_fit, _ = retrieve_polylines_from_several_images(
# #         np.dstack([warped_edges, warped_yellows, warped_whites]), draw_windows=False
# #     )

    # Average the lines
    left_lines.append(left_fit)
    if len(left_lines) > line_history_length:
        del left_lines[0]
    avg_left_fit = np.mean(left_lines, axis=0)
#     prev_left_holder[0] = avg_left_fit
    
    right_lines.append(right_fit)
    if len(right_lines) > line_history_length:
        del right_lines[0]
    avg_right_fit = np.mean(right_lines, axis=0)
#     prev_right_holder[0] = avg_right_fit
    
    # Draw the area
    dewarped = get_lane_area(warped_line_img, inv_transformation_matrix, rgb_img.shape, avg_left_fit, avg_right_fit)
    
    # Overlay
    alpha = 0.2
    combined_img = np.zeros_like(undistorted)
    cv2.addWeighted(dewarped, alpha, undistorted, 1 - alpha, 0, combined_img)
    
    # Scale lines for curvature detection 
    if scaled_lines is not None:
        scaled_left_fit, scaled_right_fit = scaled_lines

        # Average the scaled lines for curvature calculation
        scaled_left_lines.append(scaled_left_fit)
        if len(scaled_left_lines) > line_history_length:
            del scaled_left_lines[0]
        scaled_avg_left_fit = np.mean(scaled_left_lines, axis=0)

        scaled_right_lines.append(scaled_right_fit)
        if len(scaled_right_lines) > line_history_length:
            del scaled_right_lines[0]
        scaled_avg_right_fit = np.mean(scaled_right_lines, axis=0)
        
        # Select line with less variance
        y_point = 400 / 720 * YM_PER_PX  # selected y_pixel / image height
        scaled_left_fit_x = scaled_avg_left_fit[0] * y_point**2 + scaled_avg_left_fit[1] * y_point + scaled_avg_left_fit[2]
        left_line_centerpoints.append(scaled_left_fit_x)
        left_line_centerpoints = left_line_centerpoints[:-line_history_length]
        
        scaled_right_fit_x = scaled_avg_right_fit[0] * y_point**2 + scaled_avg_right_fit[1] * y_point + scaled_avg_right_fit[2]
        right_line_centerpoints.append(scaled_right_fit_x)
        right_line_centerpoints = right_line_centerpoints[:-line_history_length]
        
        # Compute the distance to the center
        gap_between_lanes = 3.7  # meters
        
        pick_left = False  # set flag to reuse result
        if np.var(left_line_centerpoints) < np.var(right_line_centerpoints):
            pick_left = True
            
        if pick_left:
            scaled_right_fit_x = scaled_left_fit_x + gap_between_lanes
        else:
            scaled_left_fit_x = scaled_right_fit_x - gap_between_lanes
        
        mid = (scaled_left_fit_x + scaled_right_fit_x) / 2
        car_offset = (np.shape(combined_img)[1] / 2 * XM_PER_PX - mid)
        car_offset_text = "Vehicle is {car_offset: .2f}m {direction_word} of center"
        
        # Draw distance to the center
        cv2.putText(
            img=combined_img,
            text=car_offset_text.format(
                car_offset=np.absolute(car_offset),
                direction_word="left" if car_offset < 0 else "right"),
            org=(50, 100),
            fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
            fontScale=2,
            color=(255,255,255),
            thickness=2,
            lineType=cv2.FILLED
        )
        
        # Compute curvature
#         left_curverad  = ((1 + (2 * scaled_left_fit[0] * y_point  + scaled_left_fit[1]) ** 2) ** 1.5)  / np.absolute(2 * scaled_left_fit[0])
#         right_curverad = ((1 + (2 * scaled_right_fit[0] * y_point + scaled_right_fit[1]) ** 2) ** 1.5) / np.absolute(2 * scaled_right_fit[0])
    
        if pick_left:
            curverad  = ((1 + (2 * scaled_avg_left_fit[0] * y_point  + scaled_avg_left_fit[1]) ** 2) ** 1.5)  / np.absolute(2 * scaled_avg_left_fit[0])
        else:
            curverad = ((1 + (2 * scaled_avg_right_fit[0] * y_point + scaled_avg_right_fit[1]) ** 2) ** 1.5) / np.absolute(2 * scaled_avg_right_fit[0])
        
#         left_curverad, right_curverad = compute_curvature(400, scaled_avg_left_fit, scaled_avg_right_fit)
        
        
        # Compute curvature radii of both lines
#         left_curverad, right_curverad = compute_curvature(677, scaled_avg_left_fit, scaled_avg_right_fit)
#         avg_curverad = (left_curverad + right_curverad) / 2
        
        cv2.putText(
            img=combined_img,
            text="Radius of Curvature = {curverad: d}(m)".format(curverad=int(curverad)),
            org=(50, 50),
            fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
            fontScale=2,
            color=(255,255,255),
            thickness=2,
            lineType=cv2.FILLED
        )
        
#     # Draw distance from middle line.
#     y_point = 400  # Farther points tend to be slightly more stable than the ones near the camera, when dashed lines are present
    
#     left_fit_x = avg_left_fit[0] * y_point**2 + avg_left_fit[1] * y_point + avg_left_fit[2]
#     left_line_centerpoints.append(left_fit_x)
#     left_line_centerpoints = left_line_centerpoints[:-line_history_length]
    
#     right_fit_x = avg_right_fit[0] * y_point**2 + avg_right_fit[1] * y_point + avg_right_fit[2]
#     right_line_centerpoints.append(right_fit_x)
#     right_line_centerpoints = right_line_centerpoints[:-line_history_length]
    
#     gap_between_lanes = 850
#     if np.var(left_line_centerpoints) < np.var(right_line_centerpoints):
#         right_fit_x = left_fit_x + gap_between_lanes
#     else:
#         left_fit_x = right_fit_x - gap_between_lanes
    
# #     left_fit_x = scaled_avg_left_fit[0] * y_point**2 + scaled_avg_left_fit[1] * y_point + scaled_avg_left_fit[2]
# #     right_fit_x = scaled_avg_right_fit[0] * y_point**2 + scaled_avg_right_fit[1] * y_point + scaled_avg_right_fit[2]

#     mid = (right_fit_x + left_fit_x) / 2
#     car_offset = (np.shape(combined_img)[1] / 2 - mid)
#     car_offset_text = "Vehicle is {car_offset: .2f}px {direction_word} of center"
    
#     cv2.putText(
#         img=combined_img,
#         text=car_offset_text.format(
#             car_offset=np.absolute(car_offset),
#             direction_word="left" if car_offset < 0 else "right"),
#         org=(50, 100),
#         fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
#         fontScale=2,
#         color=(255,255,255),
#         thickness=2,
#         lineType=cv2.FILLED
#     )
    
    return combined_img


In [81]:
video_in_folder  = os.path.join(root_folder, "data")
video_out_folder = os.path.join(root_folder, "results")
video_in_pattern = os.path.join(video_in_folder, "*.mp4")
video_in_fnames  = glob.glob(video_in_pattern)
video_out_fnames = [os.path.join(video_out_folder, os.path.basename(fname)) for fname in video_in_fnames]

only_use_videos = [2]  # Only use videos with indices specified here
use_subclip = False  # For testing

for i in range(len(video_in_fnames)):
    if only_use_videos and i not in only_use_videos:
        continue
    video_in_fname  = video_in_fnames[i]
    video_out_fname = video_out_fnames[i]
    
    clip = VideoFileClip(video_in_fname)
    if use_subclip:
        clip = clip.subclip(0, 3)
        
    LINE_HISTORY_LENGTH = 7
    left_lines = []
    right_lines = []
    prev_left_holder = [None]
    prev_right_holder = [None]
    scaled_left_lines = []
    scaled_right_lines = []
    
    # used for selecting the more stable line for curvature estimation
    left_line_centerpoints = []
    right_line_centerpoints = []
    
    processed_clip = clip.fl_image(
        lambda x: process_img(x,
                              left_lines, right_lines, LINE_HISTORY_LENGTH,
                              prev_left_holder, prev_right_holder,
                              scaled_left_lines, scaled_right_lines,
                              left_line_centerpoints, right_line_centerpoints
        )
    )

    %time processed_clip.write_videofile(video_out_fname, audio=False)
    
    clip.close()
    processed_clip.close()
    


[MoviePy] >>>> Building video D:\Linas\projects\CarND-Advanced-Lane-Lines\results\project_video.mp4
[MoviePy] Writing video D:\Linas\projects\CarND-Advanced-Lane-Lines\results\project_video.mp4



 13%|█▎        | 161/1261 [01:07<07:38,  2.40it/s]


 25%|██▌       | 321/1261 [02:14<06:32,  2.39it/s]


 38%|███▊      | 481/1261 [03:22<05:28,  2.38it/s]


 51%|█████     | 641/1261 [04:30<04:21,  2.37it/s]


 64%|██████▎   | 801/1261 [05:38<03:14,  2.37it/s]


 76%|███████▌  | 961/1261 [06:45<02:06,  2.37it/s]


 89%|████████▊ | 1119/1261 [07:50<00:59,  2.38it/s]


100%|█████████▉| 1260/1261 [08:50<00:00,  2.38it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: D:\Linas\projects\CarND-Advanced-Lane-Lines\results\project_video.mp4 

Wall time: 8min 50s


In [ ]:
# Display the videos
video_in_folder  = os.path.join(root_folder, "data")
video_out_folder = os.path.join(root_folder, "results")
video_in_pattern = os.path.join(video_in_folder, "*.mp4")
video_in_fnames  = glob.glob(video_in_pattern)
video_out_fnames = [os.path.join(video_out_folder, os.path.basename(fname)) for fname in video_in_fnames]

video_index = 0
use_subclip = False

composed_out_folder = os.path.join(root_folder, "results", "intermediate")
composed_out_fname = os.path.join(composed_out_folder, "yellow_detection_" + os.path.basename(video_out_fnames[video_index]))

original_clip = VideoFileClip(video_in_fnames[video_index]).resize(0.5)
modified_clip = VideoFileClip(video_out_fnames[video_index]).resize(0.5)
if use_subclip:
    original_clip = original_clip.subclip(0, 10)
    modified_clip = modified_clip.subclip(0, 10)
composed_clip = clips_array([[original_clip, modified_clip]])

composed_clip.write_videofile(composed_out_fname, audio=False)


In [ ]:
original_clip.close()
modified_clip.close()
composed_clip.close()

## Auxiliary Tools
This should probably be split into a separate notebook, but I need some small tools to tinker with. E.g. I need to save one frame of an image to analyze the hue and lightness value of yellow line.

In [ ]:
# video_in_name  = os.path.join(root_folder, "results", "project_video.mp4")
# video_in_name  = os.path.join(root_folder, "results", "harder_challenge_video.mp4")
video_in_name  = os.path.join(root_folder, "results", "intermediate", "channels", "s challenge_video.mp4")
out_folder = os.path.join(root_folder, "results", "intermediate", "dissected")

clip = VideoFileClip(video_in_name)

for i in range(int(clip.duration)):
    out_filename = os.path.join(out_folder, "frame_%d.png" % i)
    clip.save_frame(out_filename, t=i)
    
print("Done! Frames produced: %d" % clip.duration)
clip.close()